In [109]:
import pandas as pd 
import numpy as np 
import os
import math 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import OneHotEncoder

In [110]:
path = "."
fname = os.path.join(path, "edu_classification.csv")
df = pd.read_csv(fname)

In [111]:
df = df.drop(columns = 'Unnamed: 0')
df

,company,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,Race,Education,rowNumber,tag,salarygroup,gender
0,IBM,Software Engineer,78000,"Toronto, ON, Canada",4.0,4.0,Two Or More,Bachelor's Degree,75325,Full Stack,b. 50-100k,Male
1,EPAM Systems,Software Engineer,42000,"Kaliningrad, KN, Russia",10.0,1.0,White,Master's Degree,50462,Web Development (Front-End),a. <50k,Male
2,Five Rings,Software Engineer,330000,"New York, NY",1.0,0.0,NaN,NaN,56390,ML / AI,g. 300-350k,Female
3,Facebook,Software Engineering Manager,458000,"Seattle, WA",15.0,0.0,NaN,Master's Degree,32194,Site Reliability (SRE),j. 450-500k,Female
4,Bloomberg,Software Engineer,163000,"London, EN, United Kingdom",10.0,4.0,NaN,Master's Degree,22677,Distributed Systems (Back-End),d. 150-200k,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
53855,Wayfair,Software Engineer,114000,"Boston, MA",2.0,1.0,NaN,Master's Degree,23934,Android,c. 100-150k,Male
53856,Nvidia,Software Engineer,228000,"Santa Clara, CA",3.0,3.0,NaN,NaN,13770,ML / AI,e. 200-250k,Male
53857,ASUS,Hardware Engineer,50000,"Taipei, TP, Taiwan",6.0,5.0,NaN,Master's Degree,83294,Full Stack,a. <50k,Other
53858,Uber,Product Designer,174000,"San Francisco, CA",6.0,1.0,NaN,NaN,15505,Web Development (Front-End),d. 150-200k,Female


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53860 entries, 0 to 53859
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  53856 non-null  object 
 1   title                    53860 non-null  object 
 2   totalyearlycompensation  53860 non-null  int64  
 3   location                 53860 non-null  object 
 4   yearsofexperience        53860 non-null  float64
 5   yearsatcompany           53860 non-null  float64
 6   Race                     19373 non-null  object 
 7   Education                26128 non-null  object 
 8   rowNumber                53860 non-null  int64  
 9   tag                      53860 non-null  object 
 10  salarygroup              53860 non-null  object 
 11  gender                   53860 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 4.9+ MB


In [113]:
data = df[['title', 'totalyearlycompensation', 'yearsofexperience', 'Education', 'tag', 'gender']].copy()

In [114]:
#encode titles
titles = [[x] for x in data.title]
title_encoder = OneHotEncoder(sparse=False)
ohe_titles = title_encoder.fit_transform(titles)
ohe_title_df = pd.DataFrame(data = ohe_titles, columns = title_encoder.get_feature_names(['title']))
data = pd.concat([data, ohe_title_df], axis = 1)

In [115]:
#encode tag
tags = [[x] for x in data.tag]
tag_encoder = OneHotEncoder(sparse=False)
ohe_tags = tag_encoder.fit_transform(tags)
ohe_tag_df = pd.DataFrame(data = ohe_tags, columns = tag_encoder.get_feature_names(['tag']))
data = pd.concat([data, ohe_tag_df], axis = 1)

In [116]:
#encode gender 
gender = [[x] for x in data.gender]
gender_encoder = OneHotEncoder(sparse=False)
ohe_gender = gender_encoder.fit_transform(gender)
ohe_gender_df = pd.DataFrame(data = ohe_gender, columns = gender_encoder.get_feature_names(['gender']))
data = pd.concat([data, ohe_gender_df], axis = 1)

In [120]:
data

,title,totalyearlycompensation,yearsofexperience,Education,tag,gender,title_Business Analyst,title_Data Scientist,title_Hardware Engineer,title_Human Resources,...,tag_Other,tag_Production,tag_Security,tag_Site Reliability (SRE),tag_Testing (SDET),tag_Web Development (Front-End),tag_iOS,gender_Female,gender_Male,gender_Other
0,Software Engineer,78000,4.0,Bachelor's Degree,Full Stack,Male,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Software Engineer,42000,10.0,Master's Degree,Web Development (Front-End),Male,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,Software Engineer,330000,1.0,NaN,ML / AI,Female,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Software Engineering Manager,458000,15.0,Master's Degree,Site Reliability (SRE),Female,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Software Engineer,163000,10.0,Master's Degree,Distributed Systems (Back-End),Male,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53855,Software Engineer,114000,2.0,Master's Degree,Android,Male,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
53856,Software Engineer,228000,3.0,NaN,ML / AI,Male,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
53857,Hardware Engineer,50000,6.0,Master's Degree,Full Stack,Other,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
53858,Product Designer,174000,6.0,NaN,Web Development (Front-End),Female,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
